Aqui se toman frases en ingles y se traducen a expresiones lógicas.

Se utiliza un conjunto de reglas predefinidas para hacer coincidir las frases con expresiones lógicas.


Una **expresión regular** (o re) especifica un conjunto de cadenas que coinciden con ella; las funciones de este módulo permiten comprobar si una determinada cadena coincide con una expresión regular dada (o si una expresión regular dada coincide con una determinada cadena, que se reduce a lo mismo).

El módulo "re" tiene funciones para buscar coincidencias en una cadena.

Sustituir '{Q}' por '(?P<Q>.+??)', que significa 'coincide con 1 o más caracteres, y llámalo Q'.

La función **nombre_grupo** se utiliza en la función **Regla** para crear grupos con nombre en expresiones regulares que se utilizan para coincidir con patrones en el texto de entrada.

La función toma un patrón de expresión regular como de entrada y sustituye cualquier instancia de {} por (?P<{}>.+?), lo que crea un grupo con el nombre especificado.

Esto facilita la extracción posterior de los valores de estos grupos.

In [22]:
#el módulo tiene funciones para buscar coincidencias en una cadena
import re

In [23]:
#producir salida si la entrada coincide con algún patrón
def Rule(output, *patterns):
    return (output, [name_group(pat) + '$' for pat in patterns])

#Sustituir '{Q}' con '(?P<Q>.+?)', que significa 'coincide con 1 o más caracteres, y lo llama Q'
def name_group(pat):
    return re.sub('{(.)}', r'(?P<\1>.+?)', pat)

#Devuelve una expresión regular que coincide con w como una palabra completa no letras
def word(w):
    return r'\b' + w + r'\b' # '\b' matches at word boundary

In [24]:
rules = [
    Rule('{P} ⇒ {Q}',          'if {P} then {Q}', 'if {P}, {Q}'),
    Rule('{P} ⋁ {Q}',          'either {P} or else {Q}', 'either {P} or {Q}'),
    Rule('{P} ⋀ {Q}',          'both {P} and {Q}'),
    Rule('～{P} ⋀ ～{Q}',       'neither {P} nor {Q}'),
    Rule('～{A}{P} ⋀ ～{A}{Q}', '{A} neither {P} nor {Q}'),
    Rule('～{Q} ⇒ {P}',        '{P} unless {Q}'),
    Rule('{P} ⇒ {Q}',          '{Q} provided that {P}', '{Q} whenever {P}',
                               '{P} implies {Q}', '{P} therefore {Q}',
                               '{Q}, if {P}', '{Q} if {P}', '{P} only if {Q}'),
    Rule('{P} ⋀ {Q}',          '{P} and {Q}', '{P} but {Q}'),
    Rule('{P} ⋁ {Q}',          '{P} or else {Q}', '{P} or {Q}'),
    ]

negations = [
    (word("not"), ""),
    (word("cannot"), "can"),
    (word("can't"), "can"),
    (word("won't"), "will"),
    (word("ain't"), "is"),
    ("n't", ""), # coincidencias como parte de una palabra: didn't, couldn't, etc.
    ]

In [25]:
##Emparejar la frase con todas las reglas
#devolver la traducción lógica y la def en inglés
def match_rules(sentence, rules, defs):
    sentence = clean(sentence)
    for rule in rules:
        result = match_rule(sentence, rule, defs)
        if result:
            return result
    return match_literal(sentence, negations, defs)

In [26]:
#devuelve la transición lógica y el dict de def
#si la coincidencia tuvo éxito
def match_rule(sentence, rule, defs):
    output, patterns = rule
    for pat in patterns:
        match = re.match(pat, sentence, flags=re.I)
        if match:
            groups = match.groupdict()
            for P in sorted(groups): # Recursively apply rules to each of the matching groups
                groups[P] = match_rules(groups[P], rules, defs)[0]
            return '(' + output.format(**groups) + ')', defs


In [27]:
#si Ninguna regla coincide; Añadir nueva proposición a defs. Manejar la negación.
def match_literal(sentence, negations, defs):
    polarity = ''
    for (neg, pos) in negations:
        (sentence, n) = re.subn(neg, pos, sentence, flags=re.I)
        polarity += n * '～'
    sentence = clean(sentence)
    P = proposition_name(sentence, defs)
    defs[P] = sentence
    return polarity + P, defs


In [28]:
#Devuelve el nombre antiguo de esta sentencia, si se usaba antes, o un nombre nuevo sin usar.
def proposition_name(sentence, defs, names='PQRSTUVWXYZBCDEFGHJKLMN'):
    inverted = {defs[P]: P for P in defs}
    if sentence in inverted:
        return inverted[sentence]                      # Find previously-used name
    else:
        return next(P for P in names if P not in defs) # Use a new unused name


In [29]:
#Eliminar los espacios en blanco redundantes; tratar el apóstrofe y la coma/punto final
def clean(text):
    return ' '.join(text.split()).replace("’", "'").rstrip('.').rstrip(',')
match_rule("If today is Tuesday, I have a test in english",
           Rule('{P} ⇒ {Q}', 'if {P}, {Q}'),
           {})

sentences = ''' if today is Tuesday, I have a test in english, it is Tuesday'''.split('.')


In [ ]:
#para asegurarse de que cada str (entrada) como máximo de anchura
import textwrap
#comparar las reglas con cada frase del txt e imprimir el resultado
def logic(sentences, width=80):
    for s in map(clean, sentences):
        logic, defs = match_rules(s, rules, {})
        print('\n' + textwrap.fill('English: ' + s +'.', width), '\n\nLogic:', logic)
        for P in sorted(defs):
            print('{}: {}'.format(P, defs[P]))

logic(sentences)

# --------------------------------------------------------------------------

In [31]:
def match_rule(sentence, output, patterns, defs):
    key = (output, tuple(patterns))  # Utiliza una tupla de (salida, tupla(patrones)) como clave de caché
    if key in match_rule.cache:
        return match_rule.cache[key]
    pattern_str = r'\b' + '|'.join([f'(?P<P{i+len(patterns)}>{p})' for i, p in enumerate(output.split())] + [f'(?P<P{i}>{p})' for i, p in enumerate(patterns)]) + r'\b'  # Use unique group names for each variable in output and patterns
    groups = re.match(pattern_str, sentence)
    if groups:
        groups = groups.groupdict()
        for i, P in enumerate(output.split()):
            if f'P{i+len(patterns)}' in groups and P not in patterns:
                groups[f'P{i+len(patterns)}'], defs = match_rule(sentence, output, patterns + [groups[f'P{i+len(patterns)}']], defs)
        result = '(' + output.format(**groups) + ')', defs
    else:
        result = '', defs
    match_rule.cache[key] = result
    return result
match_rule.cache = {}  # Inicializar la caché como un diccionario vacío

def match_rules(sentence, rules, defs={}):
    key = str((tuple(rules), defs))  # Utilizar una representación de cadena de (tuple(rules), defs) como clave de caché.
    if key in match_rules.cache:
        return match_rules.cache[key]
    sentence = clean(sentence)
    for rule in rules:
        result, defs = match_rule(sentence, *rule, defs)
        if result:
            match_rules.cache[key] = result, defs
            return result, defs
    match_rules.cache[key] = None, defs
    return None, defs
match_rules.cache = {}  # Inicializar la caché como un diccionario vacío

In [32]:
def modus_ponens(P_Q, P):
    P_Q = P_Q.strip('()')
    P, Q = P_Q.split('⇒')
    if P.strip() == P and Q.strip() == Q:
        if P.strip() == P_Q.strip():
            return Q.strip()
    return None

In [33]:
def hypothetical_syllogism(P_Q, Q_R):
    P_Q = P_Q.strip('()')
    Q_R = Q_R.strip('()')
    P, Q = P_Q.split('⇒')
    Q, R = Q_R.split('⇒')
    if P.strip() == P and Q.strip() == Q and R.strip() == R:
        if Q.strip() == Q_R.strip().split('⇒')[0].strip():
            return P.strip() + ' ⇒ ' + R.strip()
    return None

In [34]:
def simplification(P_and_Q):
    P_and_Q = P_and_Q.strip('()')
    P, Q = P_and_Q.split('⋀')
    return P.strip() or Q.strip()

In [35]:
def conjunction(P, Q):
    return P.strip() + ' ⋀ ' + Q.strip()

In [36]:
def modus_tollens(P_Q, not_Q):
    P_Q = P_Q.strip('()')
    P, Q = P_Q.split('⇒')
    if P.strip() == P and Q.strip() == Q:
        if '～' + Q.strip() == not_Q.strip():
            return '～' + P.strip()
    return None

def disjunctive_syllogism(P_or_Q, not_P):
    P_or_Q = P_or_Q.strip('()')
    P, Q = P_or_Q.split('⋁')
    if '～' + P.strip() == not_P.strip():
        return Q.strip()
    elif '～' + Q.strip() == not_P.strip():
        return P.strip()
    return None

def resolution(P, not_P):
    if P.strip() == '～' + not_P.strip():
        return ''
    return None

In [37]:
def test_inference_rules(propositions):
    for i, P in enumerate(propositions):
        for j, Q in enumerate(propositions):
            if i != j:
                P_Q = match_rules(P + ' ⇒ ' + Q, rules, {})[0]
                not_Q = '～' + Q.strip()
                if P_Q and not_Q in propositions:
                    # Modus Ponens
                    result = modus_ponens(P_Q, P)
                    if result:
                        return result

                Q_R = match_rules(Q + ' ⇒ ' + P, rules, {})[0]
                if P_Q and Q_R:
                    # Silogismo hipotético
                    result = hypothetical_syllogism(P_Q, Q_R)
                    if result:
                        return result

                if '⋁' in P:
                    # Silogismo disyuntivo
                    result = disjunctive_syllogism(P, not_Q)
                    if result:
                        return result

                if '⋁' in Q:
                    # Silogismo disyuntivo
                    result = disjunctive_syllogism(Q, not_P=P)
                    if result:
                        return result

                if '⋀' in P and '⋀' in Q:
                    # Simplificacion
                    result = simplification(P + ' ⋀ ' + Q)
                    if result:
                        return result

                    # Conjuncion
                    result = conjunction(P, Q)
                    if result:
                        return result

                not_P = '～' + P.strip()
                if not_P in propositions:
                    # Modus Tollens
                    result = modus_tollens(Q_R, not_P)
                    if result:
                        return result

                    # Disjunctive Syllogism
                    result = disjunctive_syllogism(P, not_P=not_P)
                    if result:
                        return result

                    # Resolution
                    result = resolution(P, not_P)
                    if result:
                        return result

    return None

In [ ]:
#Encontrar y corregir el error
propositions = ['today is Tuesday', 'I have a test in english, it is Tuesday']
result = test_inference_rules(propositions)
print(result)